In [89]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.feature_selection import VarianceThreshold
import re
from sklearn.linear_model import Ridge


In [65]:
df=pd.read_csv('mobile_price_data.csv')
df.head()

,mobile_name,mobile_price,mobile_color,dual_sim,disp_size,resolution,os,num_cores,mp_speed,int_memory,ram,p_cam,f_cam,network,bluetooth,battery_power,mob_width,mob_height,mob_depth,mob_weight
0,"Realme Narzo 10A (So White, 32 GB)","₹8,999",So White,Dual Sim,16.56 cm (6.52 inch),1600 x 720 Pixels,Android 10,Octa Core,2 GHz,32 GB,3 GB,12MP + 2MP + 2MP,5MP Front Camera,"4G VOLTE, 4G, 3G, 2G",Yes,5000 mAh,75 mm,164.4 mm,8.95 mm,195 g
1,"Realme Narzo 10A (So Blue, 32 GB)","₹8,999",So Blue,Dual Sim,16.56 cm (6.52 inch),1600 x 720 Pixels,Android 10,Octa Core,2 GHz,32 GB,3 GB,12MP + 2MP + 2MP,5MP Front Camera,"4G VOLTE, 4G, 3G, 2G",Yes,5000 mAh,75 mm,164.4 mm,8.95 mm,195 g
2,"Realme Narzo 10A (So Blue, 64 GB)","₹9,999",So Blue,Dual Sim,16.56 cm (6.52 inch),1600 x 720 Pixels,Android 10,Octa Core,2 GHz,64 GB,4 GB,12MP + 2MP + 2MP,5MP Front Camera,"4G VOLTE, 4G, 3G, 2G",Yes,5000 mAh,75 mm,164.4 mm,8.95 mm,195 g
3,"Realme Narzo 10A (So White, 64 GB)","₹9,999",So White,Dual Sim,16.56 cm (6.52 inch),1600 x 720 Pixels,Android 10,Octa Core,2 GHz,64 GB,4 GB,12MP + 2MP + 2MP,5MP Front Camera,"4G VOLTE, 4G, 3G, 2G",Yes,5000 mAh,75 mm,164.4 mm,8.95 mm,195 g
4,"Realme Narzo 10 (That Green, 128 GB)","₹11,999",That Green,Dual Sim,16.51 cm (6.5 inch),1600 x 720 Pixels,Android 10,Octa Core,2 GHz,128 GB,4 GB,48MP + 8MP + 2MP + 2MP,16MP Front Camera,"4G VOLTE, 4G, 3G, 2G",Yes,5000 mAh,75.4 mm,164.4 mm,9 mm,199 g


In [54]:

a=df['disp_size'].str.slice(0,5).astype(dtype='float')
a


0      16.56
1      16.56
2      16.56
3      16.56
4      16.51
       ...  
454    14.48
455    14.48
456    15.82
457    16.59
458    16.59
Name: disp_size, Length: 459, dtype: float64

In [64]:
df['ram'].unique()

array([dtype('float64')], dtype=object)

In [3]:
df.shape

(459, 20)

In [6]:
df.isnull().sum()

mobile_name      0
mobile_price     0
mobile_color     0
dual_sim         0
disp_size        0
resolution       0
os               0
num_cores        0
mp_speed         0
int_memory       0
ram              0
p_cam            0
f_cam            0
network          0
bluetooth        0
battery_power    0
mob_width        0
mob_height       0
mob_depth        0
mob_weight       0
dtype: int64

In [7]:
df.drop_duplicates()
df.shape

(459, 20)

In [4]:
df.dtypes

mobile_name      object
mobile_price     object
mobile_color     object
dual_sim         object
disp_size        object
resolution       object
os               object
num_cores        object
mp_speed         object
int_memory       object
ram              object
p_cam            object
f_cam            object
network          object
bluetooth        object
battery_power    object
mob_width        object
mob_height       object
mob_depth        object
mob_weight       object
dtype: object

In [66]:
#stripping off units
df['disp_size']=df['disp_size'].str.slice(0,5).astype(dtype='float')
df['mob_width']=df['mob_width'].str.replace(' mm','',case=False).astype(dtype='float')
df['mobile_price']=df['mobile_price'].str.replace('₹','')
df['mobile_price']=df['mobile_price'].str.replace(',','').astype(dtype='float')
df['mp_speed']=df['mp_speed'].str.replace(' GHz','',case=False).astype(dtype='float')
df['int_memory']=df['int_memory'].str.replace(' GB','',case=False).astype(dtype='float')
df['ram']=df['ram'].str.replace(' GB','',case=False).astype(dtype='float')
df['battery_power']=df['battery_power'].str.replace(' mAh','',case=False).astype(dtype='float')
df['mob_height']=df['mob_height'].str.replace(' mm','',case=False).astype(dtype='float')
df['mob_depth']=df['mob_depth'].str.replace(' mm','',case=False).astype(dtype='float')
df['mob_weight']=df['mob_weight'].str.replace(' g','',case=False).astype(dtype='float')
df.dtypes

mobile_name       object
mobile_price     float64
mobile_color      object
dual_sim          object
disp_size        float64
resolution        object
os                object
num_cores         object
mp_speed         float64
int_memory       float64
ram              float64
p_cam             object
f_cam             object
network           object
bluetooth         object
battery_power    float64
mob_width        float64
mob_height       float64
mob_depth        float64
mob_weight       float64
dtype: object

In [67]:
df.head()

,mobile_name,mobile_price,mobile_color,dual_sim,disp_size,resolution,os,num_cores,mp_speed,int_memory,ram,p_cam,f_cam,network,bluetooth,battery_power,mob_width,mob_height,mob_depth,mob_weight
0,"Realme Narzo 10A (So White, 32 GB)",8999.0,So White,Dual Sim,16.56,1600 x 720 Pixels,Android 10,Octa Core,2.0,32.0,3.0,12MP + 2MP + 2MP,5MP Front Camera,"4G VOLTE, 4G, 3G, 2G",Yes,5000.0,75.0,164.4,8.95,195.0
1,"Realme Narzo 10A (So Blue, 32 GB)",8999.0,So Blue,Dual Sim,16.56,1600 x 720 Pixels,Android 10,Octa Core,2.0,32.0,3.0,12MP + 2MP + 2MP,5MP Front Camera,"4G VOLTE, 4G, 3G, 2G",Yes,5000.0,75.0,164.4,8.95,195.0
2,"Realme Narzo 10A (So Blue, 64 GB)",9999.0,So Blue,Dual Sim,16.56,1600 x 720 Pixels,Android 10,Octa Core,2.0,64.0,4.0,12MP + 2MP + 2MP,5MP Front Camera,"4G VOLTE, 4G, 3G, 2G",Yes,5000.0,75.0,164.4,8.95,195.0
3,"Realme Narzo 10A (So White, 64 GB)",9999.0,So White,Dual Sim,16.56,1600 x 720 Pixels,Android 10,Octa Core,2.0,64.0,4.0,12MP + 2MP + 2MP,5MP Front Camera,"4G VOLTE, 4G, 3G, 2G",Yes,5000.0,75.0,164.4,8.95,195.0
4,"Realme Narzo 10 (That Green, 128 GB)",11999.0,That Green,Dual Sim,16.51,1600 x 720 Pixels,Android 10,Octa Core,2.0,128.0,4.0,48MP + 8MP + 2MP + 2MP,16MP Front Camera,"4G VOLTE, 4G, 3G, 2G",Yes,5000.0,75.4,164.4,9.00,199.0


In [79]:
x=df.drop('mobile_price',axis=1)
y=df['mobile_price']
x.dtypes


mobile_name       object
mobile_color      object
dual_sim          object
disp_size        float64
resolution        object
os                object
num_cores         object
mp_speed         float64
int_memory       float64
ram              float64
p_cam             object
f_cam             object
network           object
bluetooth         object
battery_power    float64
mob_width        float64
mob_height       float64
mob_depth        float64
mob_weight       float64
dtype: object

In [80]:
trf1=ColumnTransformer([('encoder',OneHotEncoder(sparse=False),[0,1,2,4,5,6,10,11,12,13])],remainder='passthrough')

In [81]:
x=trf1.fit_transform(x)


array([[  0.  ,   0.  ,   0.  , ..., 164.4 ,   8.95, 195.  ],
       [  0.  ,   0.  ,   0.  , ..., 164.4 ,   8.95, 195.  ],
       [  0.  ,   0.  ,   0.  , ..., 164.4 ,   8.95, 195.  ],
       ...,
       [  0.  ,   0.  ,   0.  , ..., 156.  ,   7.8 , 158.  ],
       [  0.  ,   0.  ,   0.  , ..., 162.  ,   8.3 , 188.  ],
       [  0.  ,   0.  ,   0.  , ..., 162.  ,   8.3 , 188.  ]])

In [82]:
scaler=StandardScaler()
x=scaler.fit_transform(x)

In [95]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.15,random_state=1)

In [96]:
model=LinearRegression()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)

In [97]:
#training and testing score
print('Training score', model.score(x_train,y_train))
print('Testing score', model.score(x_test,y_test))


Training score 0.9822027438459279
Testing score -1.1996488627115932e+27


In [86]:
print(r2_score(y_test,y_pred))

-1.1996488627115932e+27
